In [84]:
pwd

'/Users/leilajavanmardi/Desktop/Leila/Coding_IronHack/Data_Analytics_Bootcamp/week3/project1/notebooks'

In [85]:
import numpy as np
import pandas as pd
import os
import re

In [86]:
path='/Users/leilajavanmardi/Desktop/Leila/Coding_IronHack/Data_Analytics_Bootcamp/week3/project1/data/raw/Uncleaned_DS_jobs.csv'
df = pd.read_csv(path)
shape=df.shape
print(f'The original data set has {shape[0]} rows and {shape[1]} columns')

The original data set has 672 rows and 15 columns


In [87]:
print('\nThe first 3 rows of the original dataset\n')
display(df.head(3))
print('\n\nThe last 3 rows of the original dataset\n')
df.tail(3)


The first 3 rows of the original dataset



,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1




The last 3 rows of the original dataset



,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
669,669,Data Scientist,$105K-$167K (Glassdoor est.),Join a thriving company that is changing the w...,-1.0,AccessHope,"Irwindale, CA",-1,-1,-1,-1,-1,-1,-1,-1
670,670,Data Scientist,$105K-$167K (Glassdoor est.),100 Remote Opportunity As an AINLP Data Scient...,5.0,ChaTeck Incorporated\n5.0,"San Francisco, CA","Santa Clara, CA",1 to 50 employees,-1,Company - Private,Advertising & Marketing,Business Services,$1 to $5 million (USD),-1
671,671,Data Scientist,$105K-$167K (Glassdoor est.),Description\n\nThe Data Scientist will be part...,2.7,1-800-Flowers\n2.7,"New York, NY","Carle Place, NY",1001 to 5000 employees,1976,Company - Public,Wholesale,Business Services,$1 to $2 billion (USD),-1


In [88]:
col_df=df.columns
null_values_origin=df.isna().sum()
for col in col_df:
    print(f' the missing values of the column {col} : {null_values_origin.loc[col]}')

 the missing values of the column index : 0
 the missing values of the column Job Title : 0
 the missing values of the column Salary Estimate : 0
 the missing values of the column Job Description : 0
 the missing values of the column Rating : 0
 the missing values of the column Company Name : 0
 the missing values of the column Location : 0
 the missing values of the column Headquarters : 0
 the missing values of the column Size : 0
 the missing values of the column Founded : 0
 the missing values of the column Type of ownership : 0
 the missing values of the column Industry : 0
 the missing values of the column Sector : 0
 the missing values of the column Revenue : 0
 the missing values of the column Competitors : 0


In [89]:
type=df.dtypes
for col in col_df:
    print(f' the type of the column {col} is: {type.loc[col]}')

 the type of the column index is: int64
 the type of the column Job Title is: object
 the type of the column Salary Estimate is: object
 the type of the column Job Description is: object
 the type of the column Rating is: float64
 the type of the column Company Name is: object
 the type of the column Location is: object
 the type of the column Headquarters is: object
 the type of the column Size is: object
 the type of the column Founded is: int64
 the type of the column Type of ownership is: object
 the type of the column Industry is: object
 the type of the column Sector is: object
 the type of the column Revenue is: object
 the type of the column Competitors is: object


In [90]:
df.nunique()

index                672
Job Title            172
Salary Estimate       30
Job Description      489
Rating                32
Company Name         432
Location             207
Headquarters         229
Size                   9
Founded              103
Type of ownership     13
Industry              58
Sector                23
Revenue               14
Competitors          108
dtype: int64

In [91]:
df.Headquarters.unique()

array(['New York, NY', 'Herndon, VA', 'Boston, MA',
       'Bad Ragaz, Switzerland', 'Santa Barbara, CA',
       'Basel, Switzerland', 'Bedford, MA', 'Mountain View, CA',
       'Chicago, IL', 'Mc Lean, VA', 'Saint Louis, MO', 'Richland, WA',
       'Northbrook, IL', 'Princeton, NJ', 'Mays Landing, NJ',
       'Washington, DC', 'Memphis, TN', 'Tempe, AZ', 'Reading, PA',
       'San Francisco, CA', 'Menlo Park, CA', 'Atlanta, GA',
       'Orlando, FL', 'Lexington, MA', 'Falls Church, VA',
       'Sheboygan, WI', 'Seattle, WA', 'Bellevue, WA', 'Lincoln, NE',
       'Chantilly, VA', 'Santa Monica, CA', 'Ewing, NJ',
       'South San Francisco, CA', 'Palo Alto, CA', 'Singapore, Singapore',
       'Cambridge, MA', 'OSAKA, Japan', 'Santa Clara, CA', 'Vienna, VA',
       'New Orleans, LA', 'Akron, OH', 'Zurich, Switzerland',
       'Woburn, MA', 'Carson, CA', 'Coral Gables, FL', 'San Ramon, CA',
       'Brisbane, CA', 'Winter Park, FL', 'San Rafael, CA',
       'Deerfield, IL', 'Ipswich, MA',

In [92]:
df['Job Title'].unique()

array(['Sr Data Scientist', 'Data Scientist',
       'Data Scientist / Machine Learning Expert',
       'Staff Data Scientist - Analytics',
       'Data Scientist - Statistics, Early Career', 'Data Modeler',
       'Experienced Data Scientist', 'Data Scientist - Contract',
       'Data Analyst II', 'Medical Lab Scientist',
       'Data Scientist/Machine Learning', 'Human Factors Scientist',
       'Business Intelligence Analyst I- Data Insights',
       'Data Scientist - Risk', 'Data Scientist-Human Resources',
       'Senior Research Statistician- Data Scientist', 'Data Engineer',
       'Associate Data Scientist', 'Business Intelligence Analyst',
       'Senior Analyst/Data Scientist', 'Data Analyst',
       'Machine Learning Engineer', 'Data Analyst I',
       'Scientist - Molecular Biology',
       'Computational Scientist, Machine Learning',
       'Senior Data Scientist', 'Jr. Data Engineer',
       'E-Commerce Data Analyst', 'Data Analytics Engineer',
       'Product Data Scient

In [93]:
df.Industry.unique()

array(['Insurance Carriers', 'Research & Development', 'Consulting',
       'Electrical & Electronic Manufacturing', 'Advertising & Marketing',
       'Computer Hardware & Software', 'Biotech & Pharmaceuticals',
       'Consumer Electronics & Appliances Stores',
       'Enterprise Software & Network Solutions', 'IT Services', 'Energy',
       'Chemical Manufacturing', 'Federal Agencies', 'Internet',
       'Health Care Services & Hospitals',
       'Investment Banking & Asset Management', 'Aerospace & Defense',
       'Utilities', '-1', 'Express Delivery Services',
       'Staffing & Outsourcing', 'Insurance Agencies & Brokerages',
       'Consumer Products Manufacturing', 'Industrial Manufacturing',
       'Food & Beverage Manufacturing', 'Banks & Credit Unions',
       'Video Games', 'Shipping', 'Telecommunications Services',
       'Lending', 'Cable, Internet & Telephone Providers', 'Real Estate',
       'Venture Capital & Private Equity', 'Miscellaneous Manufacturing',
       'Oil 

In [94]:
df.Industry.value_counts()

Industry
-1                                          71
Biotech & Pharmaceuticals                   66
IT Services                                 61
Computer Hardware & Software                57
Aerospace & Defense                         46
Enterprise Software & Network Solutions     43
Consulting                                  38
Staffing & Outsourcing                      36
Insurance Carriers                          28
Internet                                    27
Advertising & Marketing                     23
Health Care Services & Hospitals            21
Research & Development                      17
Federal Agencies                            16
Investment Banking & Asset Management       13
Banks & Credit Unions                        8
Lending                                      8
Energy                                       5
Consumer Products Manufacturing              5
Telecommunications Services                  5
Insurance Agencies & Brokerages              4
Food

In [95]:
def general_cleaning(df: pd.DataFrame):
    df_2=df.copy()
    df_2.columns= df_2.columns.str.lower().str.replace(' ','_').str.strip()
    df_2.drop_duplicates(inplace=True)
    df_2.reset_index(drop=True,inplace=True)
    return df_2

In [96]:
df_clean=general_cleaning(df)
df_clean.shape
print(f'After removing duplicated, the data set has {df_clean.shape[0]} rows and {df_clean.shape[1]} columns')

After removing duplicated, the data set has 672 rows and 15 columns


In [97]:
print('replacing the -1 values with numpy.nan to provide the missing indicator')
df_clean.replace({-1:np.nan,'-1': np.nan}, inplace=True)

replacing the -1 values with numpy.nan to provide the missing indicator


In [98]:
columns=df_clean.columns
null_values=df_clean.isna().sum()
for col in columns:
    print(f' the missing values of the column {col} : {null_values.loc[col]}')

 the missing values of the column index : 0
 the missing values of the column job_title : 0
 the missing values of the column salary_estimate : 0
 the missing values of the column job_description : 0
 the missing values of the column rating : 50
 the missing values of the column company_name : 0
 the missing values of the column location : 0
 the missing values of the column headquarters : 31
 the missing values of the column size : 27
 the missing values of the column founded : 118
 the missing values of the column type_of_ownership : 27
 the missing values of the column industry : 71
 the missing values of the column sector : 71
 the missing values of the column revenue : 27
 the missing values of the column competitors : 501


In [99]:
def cleaning_job_title(job_title):
    
    '''
    cleaning the job titles colmun. column name as paramter , uses regex 
    1- casting all the value to a string, cleaning the names
    2-finding the repeated pattern and replacing the values with the dic_values
    '''
    
    #1
    job_title = str(job_title)
    job_title=job_title.strip().lower()
    job_title=job_title.replace(' ','_')

    #2
    patterns = {
    r'.*senior_data.*': 'senior_data_scientist',
    r'(?<!senior_).*data_scienti.*': 'data_scientist',
    r'.*data_analy.*': 'data_analyst',
    r'.*machine_lear.*': 'machine_learning_engineer',
    r'(?i).*data_engine.*': 'data_engineer',
    r'(?i).*manage.*':'data_science_analytics_leadership',
    r'(?i).*computational.*':'computational_scientist',
    
    #r'(?i).+Scientist.+':
    }

    others='others'
    for pattern, replacement in patterns.items():
        if re.match(pattern,job_title):
            return replacement
  
    return  others

In [100]:
df_clean['job_cleaned']=df_clean.job_title.apply(cleaning_job_title)
display(df_clean.head(4))
print(f'the Jobs are now categorized in the follwoing {df_clean.job_cleaned.nunique()} groups:')
df_clean.job_cleaned.unique()

,index,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,job_cleaned
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993.0,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",data_scientist
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968.0,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),NaN,data_scientist
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981.0,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),NaN,data_scientist
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000.0,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",data_scientist


the Jobs are now categorized in the follwoing 8 groups:


array(['data_scientist', 'others', 'data_analyst', 'data_engineer',
       'machine_learning_engineer', 'senior_data_scientist',
       'data_science_analytics_leadership', 'computational_scientist'],
      dtype=object)

In [102]:
def cleaning_industry(industry):
    
    '''
    cleaning the industry colmun. column name as paramter , uses regex 
    1- casting all the value to a string, cleaning the names
    2-finding the repeated pattern and replacing the values with the dic_values
    '''
    
    #1
    industry = str(industry)
    industry = industry.strip().lower().replace(' ','_')

    patterns = {
    r'.*IT.*':'IT_services',
    r'.*computer_hardware.*|.*game.*':'computer_hardware_software',
    r'.*Aeros.*':'aerospace_defense',
    r'.*enterprise_soft.*':'enterprise_software_network_solutions ',
    r'.*consul.*':'consulting',
    r'.* pharmace.*|.*health.*': 'healthcare_and_pharmaceuticals',
    r'.*invest.*|.*bank.*|.*capital.*|.*Financ.*':'finance_and_banking',
    r'.*manufac.*':'manufacturing',
    r'.*energy.*|.*utilit.*|.*oil.*':'energy_and_utilities',
    r'.*federal.*|.*regional.*|social_ass.*':'government_public_sector',
    r'.*ship.*|.*transpor.*|rail.*':'transportation_logistics',
    r'.*real_es.*|.*constrcu.*':'real_estate_construction',
    r'.*telecommun.*':'telecommunications_services',
    r'.*internet.*':'internet_telephone_providers',
    r'.*insuranc.*':'insurance_agencies',
    r'.*adverti.*':'advertising_marketing',
    r'.*staffin.*':'staffing_outsourcing'
    }

    others='others'
    for pattern, replacement in patterns.items():
        if re.match(pattern,industry):
            return replacement

    return  others

In [103]:
df_clean['industry_cleaned']=df_clean.industry.apply(cleaning_industry)
display(df_clean.head(4))
print(f'the industries are now categorized in the follwoing {df_clean.industry_cleaned.nunique()} groups:')
df_clean.industry_cleaned.unique()

,index,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,industry,sector,revenue,competitors,job_cleaned,industry_cleaned
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993.0,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",data_scientist,insurance_agencies
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968.0,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),NaN,data_scientist,others
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981.0,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),NaN,data_scientist,consulting
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000.0,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",data_scientist,manufacturing


the industries are now categorized in the follwoing 16 groups:


array(['insurance_agencies', 'others', 'consulting', 'manufacturing',
       'advertising_marketing', 'computer_hardware_software',
       'enterprise_software_network_solutions ', 'energy_and_utilities',
       'government_public_sector', 'internet_telephone_providers',
       'healthcare_and_pharmaceuticals', 'finance_and_banking',
       'staffing_outsourcing', 'transportation_logistics',
       'telecommunications_services', 'real_estate_construction'],
      dtype=object)